<a href="https://colab.research.google.com/github/Dohwon/AI-Agent-Project/blob/main/%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D%EA%B8%B0_%EA%B8%B0%EB%B0%98_%EC%82%AC%EC%9A%A9%EC%9E%90_%EB%B0%9C%ED%99%94_%EC%9C%A0%EC%82%AC%EB%8F%84_%EB%B6%84%EC%84%9D(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 패키지 설치 (처음 한 번만 실행)

!pip install openpyxl
!pip install xlsxwriter
!apt-get install -y openjdk-11-jdk  #konlpy 위한 jdk
!pip install konlpy
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre
  x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk
  openjdk-11-jre x11-utils
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 5,367 kB of archives.
After this operation, 15.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dej

In [ ]:
# 실행 환경 체크
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

file_path = None
file_name = None
df = None

In [ ]:
import os
import platform

# 0. Colab 또는 Linux에서만 JAVA_HOME 설정
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    # JAVA 설치 및 환경 설정 (최초 1회만)
    !apt-get install -y openjdk-11-jdk
    !pip install konlpy
    !pip install openpyxl xlsxwriter
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
else:
    print("Jupyter 환경 감지 - JAVA_HOME 설정 건너뜀")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.28+6-1ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# 1. 피벗 테이블 불러오기
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter


# 미리 https://colab.research.google.com/drive/1peAYF3cgCnjHclVh7MRFFXAvqkZrBM8E?usp=sharing 에서 학습데이터 기반 피벗테이블 생성해오기
# 파일업로드 - Colab인 경우
if is_colab():
    from google.colab import files
    uploaded = files.upload()

    if uploaded:
        file_name = list(uploaded.keys())[0]
        file_path = file_name
        pivot_table = pd.read_excel(file_name, sheet_name="정규화 단어 피벗", index_col=0)
        pivot_token_table = pd.read_excel(file_name, sheet_name="전체 토큰 피벗", index_col=0)
        print(f"✅ Colab에서 '{file_name}' 업로드 완료!")

# 파일업로드 - Jupyter인 경우
else:
    file_path = "C:/Users/dowon/Desktop/학습데이터_싱글턴_인텐트별_정규화빈도_20250801 (1).xlsx"  # ← 너의 엑셀 파일명으로 바꿔줘!
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    pivot_table = pd.read_excel(file_path)
    pivot_token_table = pd.read_excel(file_path)

    print("Jupyter 환경: '{file_name}' 업로드 완료!")

Saving 학습데이터_싱글턴 (1)_인텐트별_정규화빈도_20250801.xlsx to 학습데이터_싱글턴 (1)_인텐트별_정규화빈도_20250801.xlsx
✅ Colab에서 '학습데이터_싱글턴 (1)_인텐트별_정규화빈도_20250801.xlsx' 업로드 완료!


In [ ]:
# 2. 사용자 문장 입력창 (Jupyter/Colab 겸용)
user_input = input("🙇‍♀️ 새로운 문장을 입력하세요: ")

🙇‍♀️ 새로운 문장을 입력하세요: 나이트모드 해줘


In [ ]:
### 3-1. 사용자 문장 원형 정규화
okt = Okt()

user_input_stem = okt.pos(user_input, stem=True)
user_input_norm = ""

for word, pos in user_input_stem:
    if pos in ["Noun", "Verb", "Adjective"]:
        user_input_norm += f"{word}({pos}) "

user_input_norm = user_input_norm.strip()
print("✅ 원형 기반 입력 토큰:", user_input_norm)

✅ 원형 기반 입력 토큰: 나이트(Noun) 모드(Noun) 해주다(Verb)


In [ ]:
### 3-2. 사용자 문장 전체 토큰 추출
user_tokens = okt.pos(user_input, stem=False)
user_token_str = " ".join([f"{word}({pos})" for word, pos in user_tokens])

In [ ]:
### 4-1. TF-IDF 원형 토큰 기반
intent_labels = pivot_table.columns.tolist()
intent_corpus_lemma = []
for intent in intent_labels:
    word_list = []
    for word, count in pivot_table[intent].items():
        word_list.extend([word] * int(count))
    intent_corpus_lemma.append(" ".join(word_list))

vectorizer_lemma = TfidfVectorizer(tokenizer=lambda x: x.split(), token_pattern=None)
vectorizer_lemma.fit(intent_corpus_lemma)
user_vector_lemma = vectorizer_lemma.transform([user_input_norm]).toarray()[0]
intent_matrix_lemma = vectorizer_lemma.transform(intent_corpus_lemma).toarray()

cosine_scores_lemma = cosine_similarity([user_vector_lemma], intent_matrix_lemma)[0]
results_lemma = pd.DataFrame({
    "인텐트": intent_labels,
    "Cosine 유사도": cosine_scores_lemma
}).sort_values(by="Cosine 유사도", ascending=False).reset_index(drop=True)

In [ ]:
### 4-2. TF-IDF 전체 토큰 기반
intent_corpus_token = []
for intent in intent_labels:
    word_list = []
    for word, count in pivot_token_table[intent].items():
        word_list.extend([word] * int(count))
    intent_corpus_token.append(" ".join(word_list))

vectorizer_token = TfidfVectorizer(tokenizer=lambda x: x.split(), token_pattern=None)
vectorizer_token.fit(intent_corpus_token)
user_vector_token = vectorizer_token.transform([user_token_str]).toarray()[0]
intent_matrix_token = vectorizer_token.transform(intent_corpus_token).toarray()

cosine_scores_token = cosine_similarity([user_vector_token], intent_matrix_token)[0]
results_token = pd.DataFrame({
    "인텐트": intent_labels,
    "Cosine 유사도": cosine_scores_token
}).sort_values(by="Cosine 유사도", ascending=False).reset_index(drop=True)

In [ ]:
### 5. 결과 출력
print("\n✅ 입력 문장:", user_input)
print("\n✅ 원형 기반 입력 토큰:", user_input_norm)
display(results_lemma.head(10))

print("\n✅ 전체 토큰 기반 입력 토큰:", user_token_str)
display(results_token.head(10))


✅ 입력 문장: 나이트모드 해줘

✅ 원형 기반 입력 토큰: 나이트(Noun) 모드(Noun) 해주다(Verb)


,인텐트,Cosine 유사도
0,<sk_100>(type=3)<sk_end>,0.739386
1,<sk_28>(enable=False)<sk_end>,0.618297
2,<sk_28>(enable=True)<sk_end>,0.448211
3,<sk_108>(type=3)<sk_end>,0.419942
4,<sk_100>(type=1)<sk_end>,0.278682
5,<sk_33>(enable=False)<sk_end>,0.172276
6,<sk_33>(enable=True)<sk_end>,0.156016
7,<sk_108>(type=2)<sk_end>,0.139944
8,<sk_100>(type=2)<sk_end>,0.136576
9,"<sk_40>(enable=1, type=2)<sk_end>",0.135986



✅ 전체 토큰 기반 입력 토큰: 나이트(Noun) 모드(Noun) 해줘(Verb)


,인텐트,Cosine 유사도
0,<sk_100>(type=3)<sk_end>,0.919806
1,<sk_28>(enable=False)<sk_end>,0.795386
2,<sk_28>(enable=True)<sk_end>,0.577305
3,<sk_108>(type=3)<sk_end>,0.534268
4,<sk_33>(enable=False)<sk_end>,0.206456
5,<sk_33>(enable=True)<sk_end>,0.197289
6,<sk_108>(type=2)<sk_end>,0.179130
7,<sk_100>(type=2)<sk_end>,0.169690
8,<sk_100>(type=10)<sk_end>,0.165913
9,"<sk_40>(enable=0, type=2)<sk_end>",0.157322


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 3. 인텐트별 코퍼스 생성 (pivot_table: 원형 기반 단어 빈도 피벗)
intent_labels = pivot_table.columns.tolist()
intent_corpus = []
for intent in intent_labels:
    word_list = []
    for word, count in pivot_table[intent].items():
        word_list.extend([word] * int(count))
    intent_corpus.append(" ".join(word_list))

# 4. TF-IDF Vectorizer 훈련
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(), token_pattern=None)
tfidf_vectorizer.fit(intent_corpus)

intent_matrix_tfidf = tfidf_vectorizer.transform(intent_corpus).toarray()
user_vector_tfidf = tfidf_vectorizer.transform([user_input_norm]).toarray()

# 5. Count Vectorizer 훈련
count_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(), token_pattern=None)
count_vectorizer.fit(intent_corpus)

intent_matrix_count = count_vectorizer.transform(intent_corpus).toarray()
user_vector_count = count_vectorizer.transform([user_input_norm]).toarray()

# 6. Cosine Similarity 계산
cosine_scores_tfidf = cosine_similarity(user_vector_tfidf, intent_matrix_tfidf)[0]
cosine_scores_count = cosine_similarity(user_vector_count, intent_matrix_count)[0]

# 7. 결과 출력
results = pd.DataFrame({
    "인텐트": intent_labels,
    "Cosine 유사도 (TF-IDF)": cosine_scores_tfidf,
    "Cosine 유사도 (Count)": cosine_scores_count
}).sort_values(by="Cosine 유사도 (Count)", ascending=False).reset_index(drop=True)

print(f"\n✅ 원형 기반 입력 토큰: {user_input_norm}")
display(results.head(10))
print(f"\n✅ 가장 유사한 기능토큰 (Count 기준): {results.iloc[0]['인텐트']}")


✅ 원형 기반 입력 토큰: 나이트(Noun) 모드(Noun) 해주다(Verb)


,인텐트,Cosine 유사도 (TF-IDF),Cosine 유사도 (Count)
0,<sk_100>(type=3)<sk_end>,0.739386,0.773139
1,<sk_28>(enable=False)<sk_end>,0.618297,0.717333
2,<sk_28>(enable=True)<sk_end>,0.448211,0.616116
3,<sk_108>(type=3)<sk_end>,0.419942,0.547467
4,<sk_100>(type=1)<sk_end>,0.278682,0.512303
5,<sk_33>(enable=False)<sk_end>,0.172276,0.447454
6,<sk_33>(enable=True)<sk_end>,0.156016,0.420077
7,<sk_100>(type=2)<sk_end>,0.136576,0.382741
8,<sk_108>(type=2)<sk_end>,0.139944,0.381846
9,<sk_107>(type=6)<sk_end>,0.096277,0.323762



✅ 가장 유사한 기능토큰 (Count 기준): <sk_100>(type=3)<sk_end>
